# Landfire datasets

The two landfire datasets (1.4.0 and 2.0.0) are quite different. A major overhaul of the 2.0.0 vegetation classes was performed and they do not map directly to the older version.

This notebook will combine the two vegetation classes csv files to create a single source that `generate_topo` can use. Anytime there is an update to landfire, use this notebook to update the `landfire_veg_param.csv`.

> **NOTE** this will overwrite the `landfire_veg_param.csv` file, if changes have been make be sure to add them after running this.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Landfire files
# Change the paths to reflect where these csv files live
landfire_140 = '../landfire/landfire_140/US_140EVT_20180618/CSV_Data/LF_140EVT_09152016.csv'
landfire_200 = '../landfire/landfire_200/LF2016_EVT_200_CONUS/LF2016_EVT_200_CONUS/CSV_Data/LF16_EVT_200.csv'
params = '../basin_setup/generate_topo/vegetation/landfire_veg_param.csv'

In [ ]:
# combine the two landfire datasets using the class names
# Drop all the columns that are not needed
df140 = pd.read_csv(landfire_140) 
df200 = pd.read_csv(landfire_200)

df140 = df140[['VALUE', 'CLASSNAME']]
df200 = df200[['VALUE', 'EVT_NAME']]

df140.rename(columns={'VALUE': 'landfire140'}, inplace=True)
df200.rename(columns={'VALUE': 'landfire200'}, inplace=True)
df200

In [ ]:
# joing on the following columns
# 140 - CLASSNAME
# 200 - EVT_NAME

df_merge = df140.merge(df200, right_on='EVT_NAME', left_on='CLASSNAME')
del df_merge['EVT_NAME']
df_merge

In [ ]:
# Add classes that are unique
df140_other = df140[~df140.landfire140.isin(df_merge.landfire140)]
df200_other = df200[~df200.landfire200.isin(df_merge.landfire200)]

df200_other.rename(columns={'EVT_NAME': 'CLASSNAME'}, inplace=True)

df_full = pd.concat([df_merge, df140_other, df200_other])
df_full

In [ ]:
# Add the tau and k values to those that exist
# Since this is the first time this has been done, only the landfire 1.4.0 class id's are in there so combine on those
# As more are filled in for LF2, then may need to merge on two columns
veg = pd.read_csv(params)
veg = veg[['landfire140', 'tau', 'k']]
veg.dropna(axis=0, how='all', inplace=True)

veg_merge = df_full.merge(veg, right_on='landfire140', left_on='landfire140')
veg_merge

df = pd.concat([veg_merge, df_full[~df_full.landfire140.isin(veg_merge.landfire140)]])
df

In [ ]:
# There are many duplicate class names in both datasets, try to eliminate

df = df[(~df.landfire140.duplicated()) | (df.landfire140.isnull())]
df = df[(~df.landfire200.duplicated()) | (df.landfire200.isnull())]

df.reset_index(drop=True, inplace=True)
df

In [ ]:
# format and save
df = df[['CLASSNAME', 'landfire140', 'landfire200', 'tau', 'k']]
df.to_csv(params)